In [4]:
!pip install transformers datasets scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/kaggle/input/dataset/train_dataset.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,text,status,from,translated
0,oh my gosh,Anxiety,df1,โอ้พระเจ้า!
1,trouble sleeping confused mind restless heart ...,Anxiety,df1,นอนไม่หลับ วุ่นวายใจ กระวนกระวายใจ ทุกอย่างดูผ...
2,all wrong back off dear forward doubt stay in ...,Anxiety,df1,ทุกอย่างไม่ถูกต้อง ถอยไปเถอะ อย่าก้าวไปข้างหน้...
3,i have shifted my focus to something else but ...,Anxiety,df1,ฉันพยายามเบนความสนใจไปเรื่องอื่นแล้ว แต่ก็ยังก...
4,i am restless and restless it is been a month ...,Anxiety,df1,ฉันกระวนกระวายใจมาก มันเป็นแบบนี้มาเป็นเดือนแล...
...,...,...,...,...
104995,low testosterone after discontinuing rispredon...,schizophrenia,df3,-
104996,how did you finally accept your diagnosis i am...,schizophrenia,df3,-
104997,constantly feel like i am in a competition wit...,schizophrenia,df3,-
104998,has anyone switched over to an entirely differ...,schizophrenia,df3,-


In [7]:
from sklearn.preprocessing import LabelEncoder

# Load and encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['status'])

# Get label names and mapping
label_names = label_encoder.classes_
num_labels = len(label_names)

# Optional: Check label mappings
label_mapping = dict(zip(label_names, range(num_labels)))
print(label_mapping)

{'Anxiety': 0, 'BPD': 1, 'Normal': 2, 'bipolar': 3, 'depression': 4, 'mentalillness': 5, 'schizophrenia': 6}


In [8]:
from datasets import Dataset

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df[['text', 'label']])
dataset = dataset.train_test_split(test_size=0.2)  # Split into train/test

In [13]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"  # Use bart-base for faster fine-tuning than bart-large-mnli
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/84000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments, Trainer
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_35/2541137833.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.193400,1.121688,0.605190
1000,0.901700,1.089809,0.621571
1500,1.090200,1.012352,0.649524
2000,1.071400,1.006740,0.654286
2500,0.918900,0.963201,0.665238
3000,0.853800,0.975199,0.667667
3500,0.911800,0.922279,0.681667
4000,0.894000,0.928419,0.677238
4500,0.883200,0.907786,0.682286
5000,1.122800,0.940016,0.669714


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=31500, training_loss=0.7127068643305037, metrics={'train_runtime': 10119.4827, 'train_samples_per_second': 24.902, 'train_steps_per_second': 3.113, 'total_flos': 8346190261248000.0, 'train_loss': 0.7127068643305037, 'epoch': 3.0})

In [17]:
trainer.save_model('mental-health-classifier')

In [18]:
# Now evaluate on the test (or validation) set:
results = trainer.evaluate()

print(results)  # Show metrics like accuracy, eval_loss

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.9575809240341187, 'eval_accuracy': 0.7224761904761905, 'eval_runtime': 104.6215, 'eval_samples_per_second': 200.724, 'eval_steps_per_second': 25.09, 'epoch': 3.0}
